In [42]:
from flopth import flopth
import torch
import pandas as pd
import torchvision

In [28]:

def get_flops(model):
    dummy_inputs = torch.rand(1, 3, 512, 512)
    flops, params = flopth(model, inputs=(dummy_inputs,))
    return flops, params

In [37]:
mobilenet_paths = ["./task2/runs/mobilenet/weights/mobilenet_aug_scratch.pth", "./task2/runs/mobilenet/weights/mobilenet_aug.pth", ]
resnet_paths = ["./task2/runs/resnet18/weights/resnet18_aug_scratch.pth", "./task2/runs/resnet18/weights/resnet18_aug.pth", "./task2/runs/resnet18/weights/resnet18_no_aug_scratch.pth"]

name = []
flops_list = []
params_list = []

for mdl_path in resnet_paths:
    model = torchvision.models.resnet18()
    model.fc = torch.nn.Sequential(
                torch.nn.Linear(
                    in_features = model.fc.in_features,
                    # out_features = 1,
                    out_features = 2,
                
            )
        )
    name.append(mdl_path.split("/")[-1][:-4])
    checkpoint = torch.load(mdl_path)
    model.load_state_dict(checkpoint["model"])
    flop, params = get_flops(model)
    flops_list.append(flop)
    params_list.append(params)
    # break

for mdl_path in mobilenet_paths:
    model = torchvision.models.mobilenet_v3_large()
    model.classifier[-1] = torch.nn.Sequential(
            torch.nn.Linear(
                in_features = model.classifier[-1].in_features,
                # out_features = 1,
                out_features = 2,
                
            )
        )
    name.append(mdl_path.split("/")[-1][:-4])
    checkpoint = torch.load(mdl_path)
    model.load_state_dict(checkpoint["model"])
    flop, params = get_flops(model)
    flops_list.append(flop)
    params_list.append(params)
    # break

In [39]:
df_dict = {"Model":name, "FLOPs":flops_list, "Params":params_list}

In [44]:
df = pd.DataFrame.from_dict(df_dict)
df.head()

,Model,FLOPs,Params
0,resnet18_aug_scratch,9.51334G,11.1775M
1,resnet18_aug,9.51334G,11.1775M
2,resnet18_no_aug_scratch,9.51334G,11.1775M
3,mobilenet_aug_scratch,1.18266G,4.20459M
4,mobilenet_aug,1.18266G,4.20459M


In [46]:
print(df.to_latex())

\begin{tabular}{llll}
\toprule
{} &                    Model &     FLOPs &    Params \\
\midrule
0 &     resnet18\_aug\_scratch &  9.51334G &  11.1775M \\
1 &             resnet18\_aug &  9.51334G &  11.1775M \\
2 &  resnet18\_no\_aug\_scratch &  9.51334G &  11.1775M \\
3 &    mobilenet\_aug\_scratch &  1.18266G &  4.20459M \\
4 &            mobilenet\_aug &  1.18266G &  4.20459M \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2108419/2350534188.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex())
